<a href="https://www.kaggle.com/code/abhijitdarekar001/fine-tuning-gemma-python-code-instruction?scriptVersionId=165398952" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Table of Contents:
 
1. Installing Libraries & Loading Libraries
2. Loading and Pre-Processing Data<br>
3. Loading Gemma <br>
   a. What should be the value of target modules?<br>
   b. Initilizing LoRA<br>
   c. Training Model<br>
   d. Saving Model<br>
   e. Load and Merge Model Weights<br> 
4. Text Generation Using LoRA Model
5. Conclusion

# 1. Installing and Loading Libraries

In [1]:
!pip install -q peft trl==0.7.10 evaluate transformers==4.38.0 accelerate==0.27.2 bitsandbytes==0.42.0 datasets==2.18.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments,EvalPrediction,pipeline
from peft import LoraConfig, TaskType, get_peft_model , PeftModel
from accelerate.utils import release_memory
from accelerate import Accelerator
from datasets import Dataset
import evaluate

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from trl import SFTTrainer, setup_chat_format

import torch
import torch.nn

import os
os.environ['TOKENIZERS_PARALLELISM'] = "FALSE"
import warnings
warnings.filterwarnings("ignore")

import gc
gc.collect()

2024-03-04 12:55:16.288792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 12:55:16.288925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 12:55:16.431941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


64

 # 2. Loading & Pre-Processing Data

In [3]:
code = pd.read_csv("/kaggle/input/python-code-instruction-dataset/train.csv")
code = code.sample(frac = 0.1,random_state = 101,ignore_index=True)
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861 entries, 0 to 1860
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  1861 non-null   object
 1   input        1307 non-null   object
 2   output       1861 non-null   object
 3   prompt       1861 non-null   object
dtypes: object(4)
memory usage: 58.3+ KB


In [4]:
code.head(1)

,instruction,input,output,prompt
0,"Edit the following Python code to give the output as [400, 125, 10,1].","val = [1, 10, 125, 400]\nres = []\n\nfor v in val:\n res.append(v)","val = [1, 10, 125, 400]\nres = []\n\nfor v in reversed(val):\n res.append(v)\n \nprint(res) # Output: [400, 125, 10,1]","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nEdit the following Python code to give the output as [400, 125, 10,1].\n\n### Input:\nval = [1, 10, 125, 400]\nres = []\n\nfor v in val:\n res.append(v)\n\n### Output:\nval = [1, 10, 125, 400]\nres = []\n\nfor v in reversed(val):\n res.append(v)\n \nprint(res) # Output: [400, 125, 10,1]"


In [5]:
def preprocess_data(example):
    template = f"""Instruction:\n{example['instruction']}\n\nResponse:\n{example['output']}"""
    return template
    
code['template'] = code.apply(preprocess_data,axis=1)
dataset = Dataset.from_pandas(code[['template']]).train_test_split(test_size = 0.2)

# Example Printing
print(code['template'].iloc[1])

Instruction:
Write a Python script to print all prime numbers between two numbers, a and b (where a<b).

Response:
for num in range(a, b+1): 
   if num > 1: 
       for i in range(2, num): 
           if (num % i) == 0: 
               break
       else: 
           print(num)


Above is a sample template from created dataset.

# 3. Loading Gemma

## What should be the value of target modules?

In [6]:
"""This is the structure of Gemma Model,


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=256000, bias=False)
)"""

'This is the structure of Gemma Model,\n\n\nGemmaForCausalLM(\n  (model): GemmaModel(\n    (embed_tokens): Embedding(256000, 2048, padding_idx=0)\n    (layers): ModuleList(\n      (0-17): 18 x GemmaDecoderLayer(\n        (self_attn): GemmaSdpaAttention(\n          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)\n          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)\n          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)\n          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)\n          (rotary_emb): GemmaRotaryEmbedding()\n        )\n        (mlp): GemmaMLP(\n          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)\n          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)\n          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)\n          (act_fn): GELUActivation()\n        )\n        (input_layernorm): GemmaRMSN

When you print model Summary, you will see someting like **Model Attention Layer** in this case you see **GemmaSdpaAttention** which is attention layer of the model.<br>
In our case we have attention layers which are `q_proj`, `k_proj`, `v_proj`, `o_proj`, `rotary_emb`, `up_proj`, `gate_proj`, `down_proj`.<br>
Refer : [target-modules-for-applying-peft-lora-on-different-models](https://stackoverflow.com/questions/76768226/target-modules-for-applying-peft-lora-on-different-models)


In case of LoRA, we have to mention target_modules when defining config for taining a model.
Here are possible values of `target_modeuls` 
1. `target_modules = "all-linear"`<br>
    This is a convenient shortcut that tells the system to target all linear layers within the pre-trained model for adaptation. This includes the modules you saw previously like `q_proj`, `k_proj`, `v_proj`, `o_proj`,`up_proj`, and `down_proj`.<br>
    Apart from these, Embedding layers and Output projection layer,
    
     - Benfits :<br>
     a. Simplicity  : it is sraightforward way to adapt a large portion of the model, potentially achieving significant memory savings.<br>
     b. Potential Performance Gains : As we are including majority of the model, this can help model to learn more efficintly which leads to better fine-tuning performanc.
     
     - Cons :<br>
     a. Over-adaptation : Adapting all linear layers can be overly aggressive, potentially leading to a loss of information from the pre-trained model and hindering performance. <br>
     b. Increased Memory Foot print : While aiming for memory efficiency, adapting all linear layers will still require some memory overhead compared to targeting specific modules.<br>
     
2. `target_modeules = ["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "gate_proj", "down_proj"]`<br>
    To over come the over-fitting we can secifying individual modules, you can explicitly list the modules you want to adapt for more control. This allows you to focus on specific areas while leaving less relevant parts untouched.<br>
    Below is the modules desciption.<br>
     - `q_proj`,`k_porj`,`v_proj` are query, key, values in attention mechinism.
     - `gate_proj` Projection layer for the gate in the attention mechanism.
     - `up_proj`, `down_proj` are Projection layers used for the feed-forward network within a transformer block.
    

## Initilizing LoRA

In [7]:
base_model = "/kaggle/input/gemma/transformers/2b-it/2"
lora_config = LoraConfig(
        lora_alpha=8, 
        lora_dropout=0.1,
        r=4,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='fp4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model_output_dir = "gemma2b"
training_args = TrainingArguments(
    output_dir=model_output_dir,                   
    num_train_epochs=1,                       
    per_device_train_batch_size=1,           
    gradient_accumulation_steps=8,      
    per_device_eval_batch_size = 1,
    gradient_checkpointing=True,           
    logging_steps=25,                        
    learning_rate=2e-4,                       
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                    
    max_steps=-1,
    warmup_ratio=0.03,                        
    group_by_length=True,
    lr_scheduler_type="cosine",             
    report_to="tensorboard",                  
    evaluation_strategy="steps"          
    )


trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    peft_config=lora_config,
    tokenizer=tokenizer,
    max_seq_length=1024,
    dataset_text_field = 'template',
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/1488 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

## Training Model

In [9]:
torch.cuda.empty_cache()
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
25,1.346400,0.968206
50,0.935000,0.917271
75,0.807400,0.863301
100,0.930500,0.849349
125,0.802100,0.832063
150,0.894900,0.824778
175,0.782300,0.821437


TrainOutput(global_step=186, training_loss=0.928324330237604, metrics={'train_runtime': 1883.6982, 'train_samples_per_second': 0.79, 'train_steps_per_second': 0.099, 'total_flos': 2673281953443840.0, 'train_loss': 0.928324330237604, 'epoch': 1.0})

## Saving Adapter

In [10]:
lora_adapter_path = "LoraAdapter"
trainer.model.save_pretrained(lora_adapter_path)

In [11]:
# Release Memory
trainer, model, = release_memory(trainer, model)
gc.collect()
torch.cuda.empty_cache()


## Load and Merge Model Weights 

In [12]:
final_model = "Final_Model"
# Loading Base Model
model = AutoModelForCausalLM.from_pretrained(base_model,device_map='auto', torch_dtype=torch.float16)

# Lodaing  Base Model with LoRA adapters.
peft_model = PeftModel.from_pretrained(model,lora_adapter_path,device_map='auto', torch_dtype=torch.float16)

# Merging and Saving Model
model = peft_model.merge_and_unload(progressbar = True)
model.save_pretrained(final_model)
tokenizer.save_pretrained(final_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unloading and merging model: 100%|██████████| 384/384 [00:00<00:00, 4823.19it/s]


('Final_Model/tokenizer_config.json',
 'Final_Model/special_tokens_map.json',
 'Final_Model/tokenizer.model',
 'Final_Model/added_tokens.json',
 'Final_Model/tokenizer.json')

In [13]:
model = release_memory(model)

# 4. Text Generation Using LoRA Model

In [14]:
peft_pipe = pipeline("text-generation",final_model, model_kwargs={"torch_dtype": torch.float16},
    device_map='auto',
    max_new_tokens=512)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
def get_output(question):
    prompt = f"Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.\n\nInstruction:\n{question}\n\nResponse:\n"
    out = peft_pipe(prompt,
     do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True)
                    
    return out

In [16]:
out = get_output("Write program to find factorial of number")
print(out[0]['generated_text'])

Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.

Instruction:
Write program to find factorial of number

Response:
# Python program to find factorial of a number

def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

# Driver code
n = 5
result = factorial(n)
print("The factorial of", n, "is", result)
# The factorial of 5 is 120

# Output:
# The factorial of 5 is 120
# 
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program will run the above code and print the output.
# The program 

In [17]:
out = get_output("write a program to explain simple iterators and generators")
print(out[0]['generated_text'])

Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.

Instruction:
write a program to explain simple iterators and generators

Response:
Sure, here's a simple Python program to explain iterators and generators:

```python
def my_iterator():
  for i in range(10):
    yield i

iterator = my_iterator()

for i in iterator:
  print(i)

# Output:
# 0
# 1
# 2
# 3
# 4
# 5
# 6
# 7
# 8
# 9
```

This program defines a generator function `my_iterator` that yields the numbers from 0 to 9. The `for` loop iterates over the generator and prints the numbers.

This program demonstrates the difference between iterators and generators. Iterators are objects that yield a single value at a time, while generators are functions that yield multiple values.

The generator function `my_iterator` is defined with the `def` keyword and the `yield` keyword. The `yield` keyword is used to return a value from the function. The `for

# 5. Conclusion

In this notebook we have fine-tuned **Gemma-2b-it** model from **HuggingFace**  for Python-Code Generation.

We have covered following topics.
- Dataset Creation
- Defining Lora-config for model. 
- Why Choosing correct target Modules are nesseary?.
- Saving and Merging Lora Adapters with Model.
- Text Generation for Lora Mode